# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

# SPARK SESSION

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

23/08/15 21:24:03 WARN Utils: Your hostname, Blade-15-Base-Model resolves to a loopback address: 127.0.1.1; using 192.168.1.231 instead (on interface wlo1)
23/08/15 21:24:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/15 21:24:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/15 21:24:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# IMOORT DATA

In [3]:
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [6]:
df.head()

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

# PREPROCESSING

In [7]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
my_cols = df.select([
 'Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Churn'])

In [9]:
my_final_data = my_cols.na.drop()

In [10]:
my_final_data.show()

+----+--------------+-----+---------+-------------------+--------------------+-----+
| Age|Total_Purchase|Years|Num_Sites|       Onboard_date|            Location|Churn|
+----+--------------+-----+---------+-------------------+--------------------+-----+
|42.0|       11066.8| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|    1|
|41.0|      11916.22|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|    1|
|38.0|      12884.75| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|    1|
|42.0|       8010.76| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|    1|
|37.0|       9191.58| 5.56|      9.0|2016-01-19 15:31:15|765 Tricia Row Ka...|    1|
|48.0|      10356.02| 5.12|      8.0|2009-03-03 23:13:37|6187 Olson Mounta...|    1|
|44.0|      11331.58| 5.23|     11.0|2016-12-05 03:35:43|4846 Savannah Roa...|    1|
|32.0|       9885.12| 6.92|      9.0|2006-03-09 14:50:20|25271 Roy Express...|    1|
|43.0|       14062.6| 5.46|     11.0|2011-09-29 05:47:23|3725 Car

In [11]:
from pyspark.sql.functions import format_number,dayofmonth,hour,dayofyear,month,year,weekofyear,date_format

In [12]:
newdf = my_final_data.withColumn("Year",year(my_final_data['Onboard_Date']))

In [13]:
newdf.show()

+----+--------------+-----+---------+-------------------+--------------------+-----+----+
| Age|Total_Purchase|Years|Num_Sites|       Onboard_date|            Location|Churn|Year|
+----+--------------+-----+---------+-------------------+--------------------+-----+----+
|42.0|       11066.8| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|    1|2013|
|41.0|      11916.22|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|    1|2013|
|38.0|      12884.75| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|    1|2016|
|42.0|       8010.76| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|    1|2014|
|37.0|       9191.58| 5.56|      9.0|2016-01-19 15:31:15|765 Tricia Row Ka...|    1|2016|
|48.0|      10356.02| 5.12|      8.0|2009-03-03 23:13:37|6187 Olson Mounta...|    1|2009|
|44.0|      11331.58| 5.23|     11.0|2016-12-05 03:35:43|4846 Savannah Roa...|    1|2016|
|32.0|       9885.12| 6.92|      9.0|2006-03-09 14:50:20|25271 Roy Express...|    1|2006|
|43.0|    

In [14]:
final_df = newdf.drop("Onboard_date")

In [15]:
final_df.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Year: integer (nullable = true)



In [16]:
final_df.head()

Row(Age=42.0, Total_Purchase=11066.8, Years=7.22, Num_Sites=8.0, Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Churn=1, Year=2013)

In [17]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import split, col

In [18]:
extract_location_udf = udf(lambda x: ' '.join(x.split()[-2:]), StringType())
final_df = final_df.withColumn("Location", extract_location_udf(col("Location")))

In [19]:
final_df.show()

+----+--------------+-----+---------+-------------+-----+----+
| Age|Total_Purchase|Years|Num_Sites|     Location|Churn|Year|
+----+--------------+-----+---------+-------------+-----+----+
|42.0|       11066.8| 7.22|      8.0|     AK 89518|    1|2013|
|41.0|      11916.22|  6.5|     11.0|     RI 17756|    1|2013|
|38.0|      12884.75| 6.67|     12.0|     DE 90114|    1|2016|
|42.0|       8010.76| 6.71|     10.0|WY 30645-4695|    1|2014|
|37.0|       9191.58| 5.56|      9.0|     MH 71730|    1|2016|
|48.0|      10356.02| 5.12|      8.0|     PR 74359|    1|2009|
|44.0|      11331.58| 5.23|     11.0|IA 87713-3460|    1|2016|
|32.0|       9885.12| 6.92|      9.0|FM 59852-6150|    1|2006|
|43.0|       14062.6| 5.46|     11.0|     MA 82059|    1|2011|
|40.0|       8066.94| 7.11|     11.0|WI 51655-7561|    1|2006|
|30.0|      11575.37| 5.22|      8.0|     AA 43432|    1|2016|
|45.0|       8771.02| 6.64|     11.0|     AA 40249|    1|2015|
|45.0|       8988.67| 4.84|     11.0|     AZ 44793|    

# PIPELINE

In [20]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, VectorIndexer

In [21]:
final_df.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Year: integer (nullable = true)



In [22]:
final_df.columns

['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'Location', 'Churn', 'Year']

In [23]:
location_indexer = StringIndexer(inputCol='Location',outputCol='LocationIndex') 
location_incoder = OneHotEncoder(inputCol='LocationIndex', outputCol='LocationVec')

In [24]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'LocationVec', 'Year'],
                            outputCol='features')

In [25]:
from pyspark.ml.classification import LogisticRegression

In [26]:
from pyspark.ml import Pipeline

In [27]:
log_reg = LogisticRegression(featuresCol='features', labelCol='Churn')

In [28]:
pipeline = Pipeline(stages=[location_indexer,
                            location_incoder,
                            assembler,
                            log_reg])

# SPLIT TRAIN TEST

In [29]:
train_data , test_data = final_df.randomSplit([0.7,0.3])

In [30]:
fit_model = pipeline.fit(train_data)

In [31]:
results = fit_model.transform(test_data)

# EVALUATION

In [33]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [35]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [36]:
AUC = eval.evaluate(results)

23/08/15 21:25:01 ERROR Executor: Exception in task 0.0 in stage 39.0 (TID 38)
org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (StringIndexerModel$$Lambda$3790/0x00000008415e3040: (string) => double).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:217)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator

Py4JJavaError: An error occurred while calling o333.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 39.0 failed 1 times, most recent failure: Lost task 0.0 in stage 39.0 (TID 38) (192.168.1.231 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (StringIndexerModel$$Lambda$3790/0x00000008415e3040: (string) => double).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:217)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala